In [1]:
import pandas as pd  
import numpy as np  
from sklearn.svm import SVR
from sklearn import metrics
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV

In [2]:
RDL = pd.read_excel("/Users/moca_lucy/Documents/大學/專題/RDL_AlCu/RDL_biweekly+four-weekly+monthly.xlsx", sheet_name="Monthly", header=0)
#x = [0, 1, 2]
#RDL.drop(RDL.columns[x], axis=1, inplace=True)

In [3]:
traindata = RDL[0:11]
testdata = RDL[11:13]
x_columns = ["X1", "X2", "X3", "X4", "X5", "X6", "X13", "X14", "X15"]
x_train = traindata[x_columns]
traindata
y_train = traindata["Y"]
x_test = testdata[x_columns]
y_test = testdata["Y"]
#testdata

In [57]:
cls = SVR(kernel="rbf", C=790)
cls.fit(x_train, y_train)
cls.score(x_test, y_test)
print("the test score of SVR:{:.3f}".format(cls.score(x_train, y_train)))
print("the test score of SVR:{:.3f}".format(cls.score(x_test, y_test)))

the test score of SVR:0.941
the test score of SVR:0.934


In [39]:
param = {'kernel':['rbf'], 'C':[*np.arange(0.1, 1, 0.1), *np.arange(1, 30, 1), *np.arange(30, 100, 5), *np.arange(100, 1000, 10)], 'gamma':('auto', 'scale')}
cls = GridSearchCV(SVR(), param)#cv cross validation C gamma scoring?
cls.fit(x_train, y_train)
print(cls.best_estimator_)
print(cls.score(x_train, y_train))
print(cls.score(x_test, y_test))

SVR(C=40)
0.8946213516765967
0.7724182094966351


In [58]:
dif = 1
best_score = 0
for gamma in ['auto', 'scale']:
    for c in [*np.arange(0.1, 1, 0.1), *np.arange(1, 30, 1), *np.arange(30, 100, 5), *np.arange(100, 1000, 10)]:
        cls = SVR(gamma=gamma,C=c)
        # 5 折交叉驗證
        #scores = cross_val_score(svm,X_trainval,y_trainval,cv=5)
        #score = scores.mean()
        cls.fit(x_train,y_train)
        score = cls.score(x_test,y_test)
        if cls.score(x_train, y_train)>0 and score>0 and np.abs(cls.score(x_train, y_train)-score < dif):
            dif = np.abs(cls.score(x_train, y_train)-score)
            best_score = cls.score(x_train, y_train)
            test_score = score
            best_parameters = {'gamma':gamma,"C":c}
 
# 使用最佳參數，構建新的模型
cls = SVR(**best_parameters)
cls.fit(x_train, y_train)

print('Best socre:{:.2f}'.format(best_score))
print('Best parameters:{}'.format(best_parameters))
print('Best score on test set:{:.2f}'.format(test_score))
model = cls
model.predict(x_test)

Best socre:0.94
Best parameters:{'gamma': 'scale', 'C': 740}
Best score on test set:0.94


array([28.98915204, 21.58732363])

In [27]:
'''parameters = {'kernel': ('linear', 'rbf'), 'C': [1, 2, 4], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4]}
clf = GridSearchCV(cls, param_grid=parameters)
grid_search = clf.fit(x_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print(grid_search.best_estimator_)'''

'parameters = {\'kernel\': (\'linear\', \'rbf\'), \'C\': [1, 2, 4], \'gamma\': [0.125, 0.25, 0.5, 1, 2, 4]}\nclf = GridSearchCV(cls, param_grid=parameters)\ngrid_search = clf.fit(x_train, y_train)\n\nprint("Best score: %0.3f" % grid_search.best_score_)\nprint(grid_search.best_estimator_)'

In [28]:
#gammas = np.logspace(-2, 1)
#gammas

In [25]:
#svr_predict = cls.predict(x_test)

In [9]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

In [ ]:
sum=0
    denom=0
    for index, value in enumerate(y_true):
        if value!=0:
            sum+=abs((y_true[index]-y_pred[index])/y_true[index])
            denom+=1
    return (sum/denom)*100

In [10]:
def mase(y_true, y_pred):
    y_num = y_true.size
    d = np.abs(np.diff(y_true)).sum()/(y_num-1)
    ret = np.abs(y_true - y_pred)
    return ret.mean()/d

In [59]:
def table(lm, x, y, w):
    y_pred = lm.predict(x)
    y_true = y
   # print(y_true[190])
    #print(np.where(np.isnan(y_pred)))
    #print(y_pred, y_true)
    MAE = round(metrics.mean_absolute_error(y_true, y_pred), 4)
    #print(metrics.mean_absolute_error(y_true, y_pred))
    MSE = round(metrics.mean_squared_error(y_true, y_pred), 4)
    RMSE = round(np.sqrt(metrics.mean_squared_error(y_true, y_pred)), 4)
    try:
        MAPE =  str(round(mape(y_true, y_pred), 4))+'%'
    except ZeroDivisionError:
        MAPE = "NAN"
    MASE = round(mase(y_true, y_pred), 4)
    errortype = ["MAE", "MSE", "RMSE", "MAPE", "MASE"]
    error = [MAE, MSE, RMSE, MAPE, MASE]
    #print(error)
    if w == "train":
        error_dict = {" ":errortype, "training":error}
    elif w == "test":
        error_dict = {" ":errortype, "testing":error}
    #error_dict = {" ":errortype, "誤差分析":error}
    pd.options.display.float_format = '{:.4f}'.format
    df = pd.DataFrame(error_dict)
    df = df.set_index(" ").rename_axis(None)
    #print(df)
    return df

df1 = table(cls, x_train, y_train, "train")
df1

,training
MAE,0.4669
MSE,0.9272
RMSE,0.9629
MAPE,2.0702%
MASE,0.0865


In [60]:
df2 = table(cls, x_test, y_test, "test")
df2

,testing
MAE,0.7991
MSE,1.2599
RMSE,1.1224
MAPE,3.987%
MASE,0.0888
